In [ ]:
from pyspark.sql import Row

from pyspark.sql.functions import from_json, col

import time
import requests
import json


def get_api_data(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return json.loads(response.text)
        else:
            print(f"Failed to retrieve data. Status Code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error occurred: {e}")
        return None

def process_api_stream(urls, spark, schema, batch_interval=60):
    #while True:
        all_data = []
        for url in urls:
            data = get_api_data(url)
            if data:
                all_data.append(data)
        
        # Convert the JSON data into Spark DataFrame
        if all_data:
            # Here we simulate rows. Adjust based on actual API response format
            df = spark.createDataFrame([Row(**item) for item in all_data], schema)
            
            # Process the DataFrame (e.g., transform, clean, etc.)
            df.show(truncate=False)

            # Save to a table (e.g., Hive, PostgreSQL, or HDFS)
            df.write \
                .format("parquet") \
                .mode("append") \
                .saveAsTable("default.APIData")  # Modify this to your table destination

        # Sleep for the batch interval (e.g., 60 seconds)
        time.sleep(batch_interval)


In [ ]:
print("Functions loaded!")